### RSA Algorithm
First we need to compute GCD to verify whether two integers are coprime or not

In [1]:
def gcd(a,b):
    if b==0:
        return a
    else:
        return gcd(b, a%b)

n1 = gcd(50,10)
n2 = gcd(99,33)
n3 = gcd(59,9)

import math
m1=math.gcd(50,10)
m2=math.gcd(99,33)
m3=math.gcd(59,9)

assert(n1==m1)
assert(n2==m2)
assert(n3==m3)

print("gcd(50,10) =",m1)
print("gcd(99,1033) =",m2)
print("gcd(59,9) =",m3)

gcd(50,10) = 10
gcd(99,1033) = 33
gcd(59,9) = 1


Generate two prime numbers p and q (which are supposed to be kept secret)

In [2]:
p = 13
q = 19
print("Our secret prime numbers p and q are: ",p ,q)

Our secret prime numbers p and q are:  13 19


In [3]:
n = p * q
print("modulus n = p * q = ", n)

modulus n = p * q =  247


Now, compute the Euler totient function (phi), for computing the modular multiplicative inverse operation, and keep this also as a secret

In [4]:
phi = (p -1)* (q -1)
print("The secret euler's function is: ", phi)

The secret euler's function is:  216


Now, let us calculate the public and private keys

In [5]:
e = 2
while(e < phi):
    if(math.gcd(e, phi) ==1):
        break
    else:
        e+= 1
print("public key (e) = ", e)

public key (e) =  5


In [8]:
d = 1
while(True):
    if((d * e) % phi == 1):
        break
    else:
        d+=1
print("private key (d) is ", d)

private key (d) is  173


In [9]:
public = (e,n)
private = (d,n)
print(f"The public key is {public} and the private key is {private}")


The public key is (5, 247) and the private key is (173, 247)


Encryption and Decryption takes place using modular eponentiation operation, and the public and private keys are complementary.

In [10]:
def encrypt(plaintext):
    return (plaintext ** e) % n

def decrypt(ciphertext):
    return (ciphertext **d) % n

msg = 9

enc_msg = encrypt(msg)
dec_msg = decrypt(enc_msg)

print("original message was: ", msg)
print("encrypted message was: ", enc_msg)
print("decrypted message was: ", dec_msg)

original message was:  9
encrypted message was:  16
decrypted message was:  9


### Symmetric key exchange using RSA

In [11]:
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.fernet import Fernet

symmetric_key = Fernet.generate_key()
print(f"Alice symmetric key: {symmetric_key}")

Alice symmetric key: b'mdMyrkCulPJ71Jmq-lwMd82JKP2mQ1UVrhtADYRgKlg='


In [15]:
bob_private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)
bob_public_key = bob_private_key.public_key()
print(f"Bob public key: {bob_public_key}")
print(f"Public numbers in bob's public key = {bob_public_key.public_numbers()}")

Bob public key: <cryptography.hazmat.backends.openssl.rsa._RSAPublicKey object at 0x00000211CBBA0AC0>
Public numbers in bob's public key = <RSAPublicNumbers(e=65537, n=24752914000791398790742875119439287064119032409006355820577039387378221719487755262029155239946930647854199640347262718320299662116773943278990069569510521122478084853830614731877506938431401360519735887285541829622225061202025623795077642727385206528375138168010083187523190716063036936962088359580634830663768483629666123066592833201793535807972105243753107614267243875418905213221130767428789300239301148052583551503043186541452280462034092311273474828598006951836919565066534999165316900960278014712031241794345307122842219902947673789483280284449556596559171385952763751879461880899192148160468791180853205557001)>


Now Alice has Bob's public key, so she can encrypt her symmetric key using this, and produce ciphertext

In [16]:
ciphertext = bob_public_key.encrypt(
    symmetric_key,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

print("ciphertext", ciphertext)

ciphertext b';\xed\xdc\xeb\x91\xa16\xf0J0+\x9d\xfc\xb0\xf0\xcbN\xca\x1b\xe5v\x8f\x0c\xbf\x06Sd\xc3y\xea\x8c\xfe\xb4H\x15\xdb\x13\xdc\xa6\xdb\x12\xed\xf8\n1\xbd\xb8\xe9i;\x980k\xe63\xed\x97\xd4\x17\xd4 w\xbe\x93Ze\xd2l\xb7aDaF\xf5\xa8\xe2\xf2G\x00<^\x96\x15)\x16\xf1\xc4\xf1.\xf0\x03"\xbc:\x8dp&\x81\x14_\x1eW\x7f|2\xdb\xa3\xe6DT&\xdf\xe3\xb4\x86\x12\xfbt\x91\xe0\xa4\xb9F%\xe9\xdax\xbcm\x96\xa2\xf5\xcd\xd9\xf5Z\x983jR\x1a 3\x85\xa9&p\x8apr\x84\xc0\xa4\xfe\xea\xe8\xac9\x10\xd1\xff\xae\x0b|\xc9@q\xfe\x998LK\xc3\xdf\xc4\xbd\xd5\x9f\x82\xe7\xf4\xc1\x94\xb4\x93\x982\xdd\'\xb9\x14,\x8e@\x94T\xf9_\x00\xb7U\x18?\xccm\xe6\x93`b\xb4`\x19\xde"%\x89\x13I\xb1\x1db\x86h\xa39\xc4\xf0+2\xe2\xae \xd1=x]\xdd\xec<\xa3\xd7\xd5\x05nf\xda\xcc\xc2\xdf\x014\xa7O\xe03\xd4'


In [20]:
decrypted_symmetric_key = bob_private_key.decrypt(
    ciphertext,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

print("decrypted key: ", decrypted_symmetric_key)
assert(decrypted_symmetric_key == symmetric_key)

decrypted key:  b'mdMyrkCulPJ71Jmq-lwMd82JKP2mQ1UVrhtADYRgKlg='


### Key Encapsulation Mechanism (KEM) using RSA

In [21]:
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives import hashes, serialization
from os import urandom

private_key_bob = rsa.generate_private_key(public_exponent=65537, key_size=2048)
public_key_bob = private_key_bob.public_key()

print("Bob's private and public keys have been created")

Bob's private and public keys have been created


In [22]:
alice_long_secret = urandom(160)
print(alice_long_secret)
print("alice's secret created")

b"\x04i)V\xd9\xb6e\xb8\xa0\xc6\x99\xd5E\xe3\xc5CL\xc6x\xe9\x17\xfe\x0bc\x11\xc2g\xfe\xa3wdm\x9dr\xf2\xb8\x15\xbc\xaa!\xe9\xcf\xba[\x92\xa6!>\x85\xde\t\x06NT\xdc\xe7I\\\x9b/\x9aQ\xe4\xaf\xce\xa3\xfe\xeeA;<\xad\xf3\xb1U\xad\xf2\x84\xac\x80\xd8)\xa7\x1c\xda\xd4K\xf5#R'\xfaKw\x16\x1e\x8f\xb4\x1ch\xd6EA\xc6\xa3\x06\x927\xe4\xd8\xde[\x04xW\xc8\xfbC\xfaw'\xec!\xff\xabL\xa8K\ru\x01\x8c\x8c%\xfb\xcf\xcf\xd7\xe0\xf7D\xc3\xe0\xdb\xe8?\xe4.\xea\xc1\xf2\xc8,fmRr\xef\x9e\x1d"
alice's secret created


In [23]:
alice_encrypted_secret = public_key_bob.encrypt(
    alice_long_secret,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

print("alice's secret encrypted")

alice's secret encrypted


In [24]:
bob_decryted_secret = private_key_bob.decrypt(
    alice_encrypted_secret,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

assert(alice_long_secret == bob_decryted_secret)
print("secrets match yayy")

secrets match yayy


In [26]:
def kdf(secret, salt):
    hkdf = HKDF(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        info=None,
        backend=None
    )

    return hkdf.derive(secret)

common_salt = urandom(16)
symmetric_key_alice = kdf(alice_long_secret, common_salt)
symmetric_key_bob = kdf(bob_decryted_secret, common_salt)

assert(symmetric_key_alice == symmetric_key_bob)
print("a symmetric key was derived successfully")


a symmetric key was derived successfully


### Digital Signatures with RSA

In [27]:
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import padding, rsa
from cryptography.hazmat.primitives.asymmetric.utils import Prehashed

bob_private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)
bob_public_key = bob_private_key.public_key()

alice_private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)
alice_public_key = alice_private_key.public_key()

print("Private and Public keys generated for Bob and Alice.")

Private and Public keys generated for Bob and Alice.


In [28]:
ciphertext = bob_public_key.encrypt(
    symmetric_key,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

print("ciphertext of symmetric key: ",ciphertext)

ciphertext of symmetric key:  b"\x8e\x99\xa95\x00T\xf9\xf7.\xa6\xa0\x03\xc6\x8e\xb3T\x8fgh\xe3\xbao\x92\x0cP\x87UHwp\x91\xd2\x17as\x82\xd8!\xd0\x98>L\x1c\x14\xf0-\x8a2g.\x158\x9a\xea\xb1{\x01\xd1\xd0\x8d\x1f\xc3is\xc6\xba\x84\xa8\xe8\xf3\x125\x11I\xfbZ{\x86\x10\x84\xdc~4W\xea\\\xe7\n\xc0\x1a-\xb8\x84\xe9\xeb\xefn\xe8M]\xea\xb3\xad\xbe\xb9\xf5\xe9b\xc4\x17\x96{e<\x15\xc0P?\xf9D\xf1\x10\xd5\xcb'(ZCd\x9e\xb5\x89\xa0\xfe<Q\x1d\xe8[\x93\x95D9\xb7\x028\x8b\xc0tO\x8d\xdd\xbd\xbe--#\x91\x00\xbe\x98\x0c\x9f\x1a\xc8\xf4\xe29e\xbf_\x9e\xe1\xca\x82\x8a\x04|\x99\xb3\x89v\xa9\xe4\x9d\x87\x97\xc9<\x15\xe6\x8dW\xf8\xb7\xfaP\xb0\x87\x8f]\x1a\xea)\x01y\xee9\x1d\xb3Y3\r\x9f\xa4p\x03\x1c\x17\x9d\xc6)\x1dBq\x7f\xea\xad\xb4D\xe8^\xf2\xc5\xfc\x16\xa2`Db\x88\x87\xba\x80\xed\x82@\xb7\x88\xb4\x89P\x8d\xed\xd7\xac"


Here we attach a digital signature, so that alice can prove that she was the sender of the message. We do this by creating a hash of the ciphertext and encrypt it using alice's private key which acts like a signature

In [29]:
digest= hashes.Hash(hashes.SHA256())
digest.update(ciphertext)
hash_to_sign = digest.finalize()

signature = alice_private_key.sign(
    hash_to_sign,
    padding.PSS(
        mgf=padding.MGF1(hashes.SHA256()),
        salt_length=padding.PSS.MAX_LENGTH
    ),
    Prehashed(hashes.SHA256())
)

print("signature: ", signature)

signature:  b"J#\x1a]\x9d6Cb68c\xed\x12\xf7\xbd\x1c%\xb2\xdeW\x7fWG|\xa6;v\x0ejP\xfa7*\xfd\x9c\xd6#V\xa2\x9fz\xe3\x95\x9b\xec\xf1\x98\xee\xde\xe9\xca;5z\x16E\xad8\x80\x83\x80\x19\x12\xf2\xf3\x9a\x13\x87 U'\x88\xfcOO\xa7\xa1\xde@\xa7\xa9\x96\xd5\xad8\x10\xc2k=\x11\xach\x15u\x1d\x8ch\x16\x03njx\x045\xc9*\x1bU\xd3\xbd{\xffB\xab\xa2t\x84\xfc\x93\xff\xad@~\x9cfV@\xaf\xf1\xb5n\xda<\xa4\xde\x1a\xb8J\xd6\x11\xba6\xffu\xca\x19\xe5\xa6\n\x9e\xd0\xbc\xa3\xa7rnC\xa4\x8aD\xe0]hr\xa6\xbd6;\x15\r\xac>\xb4\xd4\xa9\xf8\x05\x00\xc6\xcb\xc2C}\xc2\xaf\x14\x06\xbf\x943\x07\xe5\xa2\x9bVl\x92MF\xbd\xdaGDs\xf6_\xa6\t]H\x94;V~\xfa\xdb\xf0\x81\xc1\x1a\x15_?\x9c\xa2\xab\xc3W\x8a\x11\x17\xc0T\xf1T\x8bS\xd4\x80\xb5\xb9\x01\xea\x86\xf8\x80\xc0\x0f\xc0\xf0\xb2O&\xab4\x81"


In [30]:
recieved_ciphertext = ciphertext
recieved_signature = signature

print("sent ciphertext and signature")

sent ciphertext and signature


In [31]:
digest = hashes.Hash(hashes.SHA256())
digest.update(recieved_ciphertext)
hash_to_verify = digest.finalize()

print("hash to verify:", hash_to_verify)

hash to verify: b'\x80 \xf2{\x81%\x84\x8c\xb6\x1a\x9ak\xd3l\xcb\x85P\x0f\x85\xea\xc1\xba\xa7,\xb4\\!<{\xa4.t'


In [32]:
try:
    alice_public_key.verify(
        recieved_signature,
        hash_to_verify,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        Prehashed(hashes.SHA256())
    )
    print("valid signature")

except:
    print("invalid signature")

valid signature


In [33]:
decrypted_message = bob_private_key.decrypt(
     recieved_ciphertext,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

print("decrypted message: ", decrypted_message.decode())



decrypted message:  mdMyrkCulPJ71Jmq-lwMd82JKP2mQ1UVrhtADYRgKlg=


### Breaking RSA encryption
here we will have the public key, and will recover the private key from it

In [36]:
n = 247
e = 5

a = 6
assert(gcd(a, n) == 1)
print(f"{n} and {a} are coprime")

247 and 6 are coprime


In [37]:
r = 0
rem = 100
while(rem != 1):
    r+=1
    rem = (a**r) % n

print(f"period r is {r}")
assert(a**r % n == 1)
print(f"checked {a}^{r} mod {n} = 1")

period r is 36
checked 6^36 mod 247 = 1


In [38]:
f1 = int(a**(r/2) - 1)
f2 = int(a**(r/2) + 1)

print(f1)
print(f2)

101559956668415
101559956668417


In [40]:
q_found = gcd(f1, n)
print(f"one possible prime factor is: {q_found}")

p_found = gcd(f2, n)
print(f"one possible prime factor is: {p_found}")

assert(n == p_found * q_found)

one possible prime factor is: 19
one possible prime factor is: 13


In [42]:
phi_found = (p_found - 1) * (q_found - 1)
print(f"phi found is: {phi_found}")

d_found = 1
while(True):
    if((d_found * e) % phi_found == 1):
        break
    else:
        d_found+=1
print("private key found as", d_found)

phi found is: 216
private key found as 173
